In [2]:
import os
import pandas as pd
import numpy as np

import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Blink-182 song

In [47]:
# Songs
carousel = "3gVXAW2pduBl7tzjAGUVRb"
carousel_aa = sp.audio_analysis(carousel)

In [48]:
carousel_aa.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])

In [70]:
len(carousel_aa["segments"])

775

In [68]:
process_pitch(carousel_aa["segments"][0]['pitches'])

0


divido a movimentação de um ponto em 12 direções

In [38]:
sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"][0]['id']

'2e61kmae4HhbuS9hYONQ0A'

In [39]:
len(sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"])

8

In [5]:
ids_houses_of_holy = [sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"][i]['id'] for i in range(len(sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"]) - 1)]

In [20]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

def get_song_id_metrics(song_id):
    song_audio = sp.audio_analysis(song_id)
    coords = []
    # print(len(song_audio["segments"]))
    for i in range(len(song_audio["segments"])-1):
        coords.append(process_pitch(song_audio["segments"][i]['pitches']))
    return coords

def process_pitch(pitch_list):
    # notas = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
    # print(len(pitch_list))
    # try:
    min_max_list = normalize(pitch_list)
    coordinates = get_coordinates(min_max_list)
    # coordinates = coordinates[9]
    coordinates = list(np.sum(coordinates, axis=0))
        # print(coordinates)
        # pitch_dict = [coordinates[i] for i in range(len(notas)-1)]
    # except:
        # print("erro")
        # return
    # return pitch_dict
    return coordinates

def normalize(input_list):
    min_val = min(input_list)
    max_val = max(input_list)
    if max_val == min_val:
        return [0 for x in input_list]
    normalized_list = [(x - min_val) / (max_val - min_val) for x in input_list]
    return normalized_list

def get_coordinates(normalized_input):
    i = 0
    coordinates = []
    for i in range(len(normalized_input) - 1):
        # print(normalized_input[i])
        cos = round(np.cos(i * 2 * np.pi/12), 5) * normalized_input[0]
        sin = round(np.sin(i * 2 * np.pi/12), 5) * normalized_input[1]
        # print((cos, sin))
        coordinates.append((cos, sin))
    return coordinates
        
        

In [21]:
database = {}
for i,song_id in enumerate(ids_houses_of_holy):
    database[f"{i}"] = get_song_id_metrics(song_id)

In [22]:
database['1']

[(-0.0, -0.30676855895196503),
 (-0.0, -0.0825593395252838),
 (-0.0, -0.002034587995930822),
 (-0.0, -0.001002004008016032),
 (-0.0, -0.006030150753768843),
 (-0.0, -0.0020060180541624875),
 (-0.0, -0.004016064257028113),
 (-0.0, -0.0),
 (-0.0, -0.0030303030303030294),
 (-0.0, -0.03564154786150713),
 (-0.0, -0.2025862068965517),
 (-0.0, -0.01609657947686117),
 (-0.0, -0.00802407221664995),
 (-0.0, -0.011033099297893681),
 (-0.0, -0.007028112449799196),
 (-0.0, -0.015045135406218655),
 (-0.0, -0.02814070351758794),
 (-0.0, -0.05792682926829268),
 (-0.0, -0.006012024048096192),
 (-0.0, -0.0),
 (-0.0, -0.0),
 (-0.0, -0.0),
 (-0.0, -0.00518672199170125),
 (-0.0, -0.10880829015544043),
 (-0.0, -0.605927552140505),
 (-0.0, -0.025052192066805846),
 (-0.0, -0.00404040404040404),
 (-0.0, -0.022110552763819093),
 (-0.0, -0.018163471241170535),
 (-0.0, -0.008130081300813009),
 (-0.0, -0.01909547738693467),
 (-0.0, -0.022177419354838707),
 (-0.0, -0.030241935483870965),
 (-0.0, -0.0291457286432160

In [23]:
import json

with open("houses_of_the_holy.json", "w") as fp:
    json.dump(database , fp, indent = 4)